# fanout exchange

广播exchange


In [1]:
import pika
import threading

TEST_EXCHANGE_NAME = "rb_test_fanout_exchange"
connection_params = pika.ConnectionParameters(
    host="127.0.0.1",
    port=5672,
    virtual_host="/",
    credentials=pika.PlainCredentials("admin", "123456"),
)

# 生产者


In [2]:
rabbit_proudct_connection = pika.BlockingConnection(connection_params)
proudct_channel = rabbit_proudct_connection.channel()
# 如果之前的存在则删除
proudct_channel.exchange_delete(TEST_EXCHANGE_NAME)
proudct_channel.exchange_declare(TEST_EXCHANGE_NAME, "fanout")

proudct_channel.queue_declare(queue="queue_1", auto_delete=True)
proudct_channel.queue_declare(queue="queue_2", auto_delete=True)
proudct_channel.queue_declare(queue="queue_3", auto_delete=True)

proudct_channel.queue_bind(queue="queue_1", exchange=TEST_EXCHANGE_NAME)
proudct_channel.queue_bind(queue="queue_2", exchange=TEST_EXCHANGE_NAME)
proudct_channel.queue_bind(queue="queue_3", exchange=TEST_EXCHANGE_NAME)
# 不需要 binding key 和 routing key 了，因为是广播
proudct_channel.basic_publish(
    exchange=TEST_EXCHANGE_NAME,
    routing_key="",
    body=b"fanout test message,can you receive",
)

proudct_channel.close()
rabbit_proudct_connection.close()

# 消费者


In [3]:
def on_message_callback(ch, method, properties, body):
    print(body.decode("utf-8"), f"channel id: {id(ch)}")
    ch.basic_ack(delivery_tag=method.delivery_tag)


def consume_message(queue_name: str, routing_key: str):
    consumer_connection = pika.BlockingConnection(connection_params)
    consumer_channel = consumer_connection.channel()
    consumer_channel.exchange_declare(TEST_EXCHANGE_NAME, "fanout")
    consumer_channel.queue_declare(queue=queue_name, auto_delete=True)
    consumer_channel.queue_bind(queue_name, TEST_EXCHANGE_NAME, "")
    consumer_channel.basic_consume(queue_name, on_message_callback=on_message_callback)
    consumer_channel.start_consuming()


t1 = threading.Thread(target=consume_message, args=("queue_1", "我是队列1")).start()
t2 = threading.Thread(target=consume_message, args=("queue_2", "我是队列2")).start()
t3 = threading.Thread(target=consume_message, args=("queue_3", "我是队列3")).start()

fanout test message,can you receive channel id: 1444466944816
fanout test message,can you receive channel id: 1444466944912
fanout test message,can you receive channel id: 1444466944864
